In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

We want to eliminate "accidental" referrals, so filter the hop teaming data so that the transaction_count is at least 50 and the average_day_wait is less than 50.

In [ ]:
##db = sqlite3.connect('data/doc_grahp_hop.sqlite')

##chunks = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)
##hop = []
##for chunk in chunks: 
##    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)]
##    hop.append(chunk)
##    chunk.to_sql('hop', db, if_exists = 'append', index = False) 

In [ ]:
##hopp = pd.concat(hop, ignore_index = True)
##hopp


The NPPES dataset contains a large number of fields, only a few of which are relevant to this project:

In [2]:
cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name','Provider Name Prefix Text', 'Provider Name Suffix Text','Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code', 'Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10', 'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15']

nppes_chunks = pd.read_csv('data/NPPES_Data_Dissemination_February_2023/npidata_pfile_20050523-20230212.csv', usecols=cols, iterator=True, chunksize=1000)

In [3]:
state_list = ['TN', 'AR', 'NC', 'MS', 'GA']
nppes = pd.concat([chunk[chunk['Provider Business Practice Location Address State Name'].isin(state_list)] for chunk in nppes_chunks])

In [4]:
var_cols = ['Healthcare Provider Taxonomy Code', 'Healthcare Provider Primary Taxonomy Switch']

In [5]:
state_list = ['TN', 'AR', 'NC', 'MS', 'GA']
nppes = pd.DataFrame()
id_cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name','Provider Name Prefix Text', 'Provider Name Suffix Text','Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code']

for chunk in nppes_chunks:
    filtered_chunk = chunk[chunk['Provider Business Practice Location Address State Name'].isin(state_list)]
    filtered_chunk = pd.wide_to_long(filtered_chunk, var_cols, i = id_cols , j = "", sep = "_")
    
    nppes = pd.concat([nppes, filtered_chunk])

In [14]:
nppes.reset_index()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,,Healthcare Provider Taxonomy Code,Healthcare Provider Primary Taxonomy Switch
0,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009.0,1,183500000X,Y
1,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009.0,2,NaN,NaN
2,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009.0,3,NaN,NaN
3,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009.0,4,NaN,NaN
4,1750384210,1.0,NaN,GILMER,CARISSIA,NaN,NaN,NaN,PHARMD,1410 S 4TH ST,NaN,NASHVILLE,AR,718523009.0,5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,1477556611,1.0,NaN,LEWIS,ANESSA,J,NaN,NaN,M.D.,2950 VILLAGE DR,NaN,FAYETTEVILLE,NC,283043815.0,11,NaN,NaN
536,1477556611,1.0,NaN,LEWIS,ANESSA,J,NaN,NaN,M.D.,2950 VILLAGE DR,NaN,FAYETTEVILLE,NC,283043815.0,12,NaN,NaN
537,1477556611,1.0,NaN,LEWIS,ANESSA,J,NaN,NaN,M.D.,2950 VILLAGE DR,NaN,FAYETTEVILLE,NC,283043815.0,13,NaN,NaN
538,1477556611,1.0,NaN,LEWIS,ANESSA,J,NaN,NaN,M.D.,2950 VILLAGE DR,NaN,FAYETTEVILLE,NC,283043815.0,14,NaN,NaN


In [23]:
db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')

cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name','Provider Name Prefix Text', 'Provider Name Suffix Text','Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code', 'Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10', 'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15']

nppes_chunks = pd.read_csv('data/NPPES_Data_Dissemination_February_2023/npidata_pfile_20050523-20230212.csv', usecols=cols, iterator=True, chunksize=10000, nrows = 1000)
var_cols = ['Healthcare Provider Taxonomy Code', 'Healthcare Provider Primary Taxonomy Switch']

nppes_full = []

state_list = ['TN', 'AR', 'NC', 'MS', 'GA']
nppes = pd.DataFrame()
id_cols = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)','Provider First Name', 'Provider Middle Name','Provider Name Prefix Text', 'Provider Name Suffix Text','Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address','Provider Business Practice Location Address City Name','Provider Business Practice Location Address State Name','Provider Business Practice Location Address Postal Code']

for chunk in nppes_chunks:
    filtered_chunk = chunk[chunk['Provider Business Practice Location Address State Name'].isin(state_list)]
    filtered_chunk = pd.wide_to_long(filtered_chunk, var_cols, i = id_cols , j = "", sep = "_")
    
    nppes = pd.concat([nppes, filtered_chunk])

nppes.columns = nppes.columns.str.replace(" ", "")
nppes.reset_index()


nppes_full.append(nppes)
chunk.to_sql('nppes_full', db, if_exists = 'append', index = False) 

In [29]:
db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')
query = "SELECT * FROM nppes_full"

SELECT * FROM nppes_full


In [30]:
with sqlite3.connect('data/NPPES_Data_Dissemination.sqlite') as db: 
    nppes_sqlite = pd.read_sql(query, db)

In [31]:
nppes_sqlite

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,...,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15
0,1740284231,NaN,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1346245800,NaN,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,1487650776,NaN,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,1033113022,NaN,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,1043216138,NaN,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1306849922,1.0,None,PENRY,DAVID,V,None,None,APRN,"NP RESOURCES, LLC.",...,None,None,None,None,None,None,None,None,None,None
996,1124021746,1.0,None,COTTER,JOHN,BURLEY,DR.,None,MD,1270 PEACH ST,...,None,None,None,None,None,None,None,None,None,None
997,1851394472,1.0,None,PETERS,DAVID,S,None,None,MD,112 S NORTHWEST HWY,...,None,None,None,None,None,None,None,None,None,None
998,1487657003,1.0,None,CULALA,RYAN,MICHAEL,None,None,PHARM.D.,11521 NE 128TH ST,...,None,None,None,None,None,None,None,None,None,None


In [32]:
taxonomy = pd.read_csv('data/nucc_taxonomy_230.csv', nrows=100)
taxonomy

,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,"A physician who specializes in the diagnosis, ...",Source: National Uniform Claim Committee,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,An allergy and immunology physician who specia...,"Source: National Uniform Claim Committee, 2022...",Clinical & Laboratory Immunology (Allergy & Im...,Individual
...,...,...,...,...,...,...,...,...
95,207WX0120X,Allopathic & Osteopathic Physicians,Ophthalmology,Cornea and External Diseases Specialist,An ophthalmologist who specializes in diseases...,"Source: American Academy of Ophthalmology, www...",Cornea and External Diseases Specialist Physician,Individual
96,207WX0009X,Allopathic & Osteopathic Physicians,Ophthalmology,Glaucoma Specialist,An ophthalmologist who specializes in the trea...,"Source: American Academy of Ophthalmology, www...",Glaucoma Specialist (Ophthalmology) Physician,Individual
97,207WX0109X,Allopathic & Osteopathic Physicians,Ophthalmology,Neuro-ophthalmology,A neuro-ophthalmologist is a subspecialist of ...,"Source: American Academy of Ophthalmology, www...",Neuro-ophthalmology Physician,Individual
98,207WX0200X,Allopathic & Osteopathic Physicians,Ophthalmology,Ophthalmic Plastic and Reconstructive Surgery,A physician who specializes in oculofacial pla...,"Source: American Academy of Ophthalmology, 20...",Ophthalmic Plastic and Reconstructive Surgery ...,Individual
